prediction interval competition House price 

In [66]:
import pandas as pd
import numpy as np

In [67]:
df_train = pd.read_csv(r'D:\Python Code VS\kaggle_compt_data\prediction-interval-competition-ii-house-price\dataset.csv')

In [68]:
df_train.info()
df = df_train.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 47 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                200000 non-null  int64  
 1   sale_date         200000 non-null  object 
 2   sale_price        200000 non-null  int64  
 3   sale_nbr          157818 non-null  float64
 4   sale_warning      200000 non-null  object 
 5   join_status       200000 non-null  object 
 6   join_year         200000 non-null  int64  
 7   latitude          200000 non-null  float64
 8   longitude         200000 non-null  float64
 9   area              200000 non-null  int64  
 10  city              200000 non-null  object 
 11  zoning            200000 non-null  object 
 12  subdivision       182450 non-null  object 
 13  present_use       200000 non-null  int64  
 14  land_val          200000 non-null  int64  
 15  imp_val           200000 non-null  int64  
 16  year_built        20

In [69]:
df.isnull().sum()

id                      0
sale_date               0
sale_price              0
sale_nbr            42182
sale_warning            0
join_status             0
join_year               0
latitude                0
longitude               0
area                    0
city                    0
zoning                  0
subdivision         17550
present_use             0
land_val                0
imp_val                 0
year_built              0
year_reno               0
sqft_lot                0
sqft                    0
sqft_1                  0
sqft_fbsmt              0
grade                   0
fbsmt_grade             0
condition               0
stories                 0
beds                    0
bath_full               0
bath_3qtr               0
bath_half               0
garb_sqft               0
gara_sqft               0
wfnt                    0
golf                    0
greenbelt               0
noise_traffic           0
view_rainier            0
view_olympics           0
view_cascade

In [70]:
X = df.drop(columns=['sale_price'])
y = df['sale_price']

In [71]:
df.columns

Index(['id', 'sale_date', 'sale_price', 'sale_nbr', 'sale_warning',
       'join_status', 'join_year', 'latitude', 'longitude', 'area', 'city',
       'zoning', 'subdivision', 'present_use', 'land_val', 'imp_val',
       'year_built', 'year_reno', 'sqft_lot', 'sqft', 'sqft_1', 'sqft_fbsmt',
       'grade', 'fbsmt_grade', 'condition', 'stories', 'beds', 'bath_full',
       'bath_3qtr', 'bath_half', 'garb_sqft', 'gara_sqft', 'wfnt', 'golf',
       'greenbelt', 'noise_traffic', 'view_rainier', 'view_olympics',
       'view_cascades', 'view_territorial', 'view_skyline', 'view_sound',
       'view_lakewash', 'view_lakesamm', 'view_otherwater', 'view_other',
       'submarket'],
      dtype='object')

In [72]:
X['total_baths'] = X['bath_full'] + 0.75 * X['bath_3qtr'] + 0.5 * X['bath_half']


In [73]:
X['view_score'] = (
    X['view_rainier'] + X['view_olympics'] + X['view_cascades'] +
    X['view_territorial'] + X['view_skyline'] + 
    X['view_sound'] + X['view_lakewash'] + 
    X['view_lakesamm']
)

drop_cols = [
    'view_rainier', 'view_olympics', 'view_cascades', 'view_territorial',
    'view_skyline', 'view_sound', 'view_lakewash', 'view_lakesamm',
]
X = X.drop(columns=drop_cols)


In [74]:
# drop columns that are not useful for prediction
X = X.drop(columns=['id', 'view_other', 'view_otherwater', 
                    'sale_warning', 'join_year', 'latitude', 'longitude',
                    'subdivision', 'present_use', 'sqft_1', 'sqft_fbsmt',
                    'fbsmt_grade', 'bath_full', 'bath_3qtr', 'bath_half',
                    'wfnt'])


In [75]:
X.head()

,sale_date,sale_nbr,join_status,area,city,zoning,land_val,imp_val,year_built,year_reno,...,stories,beds,garb_sqft,gara_sqft,golf,greenbelt,noise_traffic,submarket,total_baths,view_score
0,2014-11-15,2.0,nochg,53,FEDERAL WAY,RS7.2,167000,372000,1975,0,...,1.0,3,0,500,0,0,0,I,1.75,0
1,1999-01-15,NaN,nochg,74,KIRKLAND,RS 8.5,1184000,598000,1962,0,...,1.0,3,0,0,0,0,0,Q,2.25,1
2,2006-08-15,1.0,nochg,30,RENTON,R-8,230000,356000,1986,0,...,2.0,3,0,480,0,0,0,K,2.50,0
3,1999-12-15,1.0,nochg,96,BURIEN,RS-7200,190000,518000,1998,0,...,2.0,4,0,530,0,0,1,G,2.50,0
4,2018-07-15,2.0,miss99,36,KING COUNTY,RA2.5,616000,1917000,1998,0,...,2.0,4,0,810,0,0,0,P,3.25,0


In [76]:
from sklearn.preprocessing import LabelEncoder

In [77]:
encoder = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = encoder.fit_transform(X[col].astype(str))
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   sale_date      200000 non-null  int64  
 1   sale_nbr       157818 non-null  float64
 2   join_status    200000 non-null  int64  
 3   area           200000 non-null  int64  
 4   city           200000 non-null  int64  
 5   zoning         200000 non-null  int64  
 6   land_val       200000 non-null  int64  
 7   imp_val        200000 non-null  int64  
 8   year_built     200000 non-null  int64  
 9   year_reno      200000 non-null  int64  
 10  sqft_lot       200000 non-null  int64  
 11  sqft           200000 non-null  int64  
 12  grade          200000 non-null  int64  
 13  condition      200000 non-null  int64  
 14  stories        200000 non-null  float64
 15  beds           200000 non-null  int64  
 16  garb_sqft      200000 non-null  int64  
 17  gara_sqft      200000 non-nul

In [78]:
from sklearn.model_selection import train_test_split


In [79]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)
# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42, shuffle=True
)


In [80]:
import xgboost as xgb

In [81]:
print(type(X_train), X_train.shape)
print(type(y_train), y_train.shape)
print(type(X_val), X_val.shape)
print(type(y_val), y_val.shape)


<class 'pandas.core.frame.DataFrame'> (128000, 24)
<class 'pandas.core.series.Series'> (128000,)
<class 'pandas.core.frame.DataFrame'> (32000, 24)
<class 'pandas.core.series.Series'> (32000,)


In [82]:
# id, prediction, lower_bound, upper_bound

model = xgb.XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=6,
    random_state=42,
    n_jobs=-1,
    early_stopping_rounds=50,
)

# Fit with early stopping on validation set
model.fit(
    X=X_train,
    y=y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)


# Predict on test set
y_pred = model.predict(X_test)

# Calculate residuals on test set
residuals = np.abs(y_test - y_pred)

# Calculate 90th percentile of residuals to define interval width
q = np.quantile(residuals, 0.9)

# Construct prediction intervals
y_pred_lower = y_pred - q
y_pred_upper = y_pred + q



[0]	validation_0-rmse:410851.62478
[1]	validation_0-rmse:407761.46065
[2]	validation_0-rmse:404701.75061
[3]	validation_0-rmse:401693.41947
[4]	validation_0-rmse:398710.00248
[5]	validation_0-rmse:395767.86181
[6]	validation_0-rmse:392861.77153
[7]	validation_0-rmse:389984.18046
[8]	validation_0-rmse:387148.94808
[9]	validation_0-rmse:384342.07262
[10]	validation_0-rmse:381572.76787
[11]	validation_0-rmse:378831.13677
[12]	validation_0-rmse:376130.62659
[13]	validation_0-rmse:373467.50057
[14]	validation_0-rmse:370835.65503
[15]	validation_0-rmse:368231.01331
[16]	validation_0-rmse:365623.03561
[17]	validation_0-rmse:363077.05925
[18]	validation_0-rmse:360511.72962
[19]	validation_0-rmse:358002.44315
[20]	validation_0-rmse:355532.75144
[21]	validation_0-rmse:353042.59204
[22]	validation_0-rmse:350573.63930
[23]	validation_0-rmse:348140.70718
[24]	validation_0-rmse:345736.52720
[25]	validation_0-rmse:343385.78737
[26]	validation_0-rmse:341036.89033
[27]	validation_0-rmse:338713.44391
[2

In [83]:
# Creating the submission DataFrame
submission = pd.DataFrame({
    'id': df_train['id'][X_test.index],
    'prediction': y_pred,
    'lower_bound': y_pred_lower,
    'upper_bound': y_pred_upper
})

In [84]:
submission['actual_price'] = y_test.values
submission

,id,prediction,lower_bound,upper_bound,actual_price
119737,119737,274109.53125,124320.028125,4.238990e+05,254920
72272,72272,404152.46875,254362.965625,5.539420e+05,400000
158154,158154,994011.87500,844222.371875,1.143801e+06,1125000
65426,65426,299471.50000,149681.996875,4.492610e+05,360000
30074,30074,749747.37500,599957.871875,8.995369e+05,861000
...,...,...,...,...,...
4174,4174,413325.15625,263535.653125,5.631147e+05,474000
91537,91537,492106.75000,342317.246875,6.418963e+05,499000
156449,156449,364924.34375,215134.840625,5.147138e+05,395000
184376,184376,901489.06250,751699.559375,1.051279e+06,800000


In [85]:
submission['true_price_fall'] = np.where(
    (submission['actual_price'] < submission['lower_bound']) | 
    (submission['actual_price'] > submission['upper_bound']),
    1, 0
)

In [86]:
submission['true_price_fall'].value_counts()

true_price_fall
0    36000
1     4000
Name: count, dtype: int64

In [87]:
df_test = pd.read_csv(r'D:\Python Code VS\kaggle_compt_data\prediction-interval-competition-ii-house-price\test.csv')

In [88]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 46 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                200000 non-null  int64  
 1   sale_date         200000 non-null  object 
 2   sale_nbr          157588 non-null  float64
 3   sale_warning      200000 non-null  object 
 4   join_status       200000 non-null  object 
 5   join_year         200000 non-null  int64  
 6   latitude          200000 non-null  float64
 7   longitude         200000 non-null  float64
 8   area              200000 non-null  int64  
 9   city              200000 non-null  object 
 10  zoning            200000 non-null  object 
 11  subdivision       182450 non-null  object 
 12  present_use       200000 non-null  int64  
 13  land_val          200000 non-null  int64  
 14  imp_val           200000 non-null  int64  
 15  year_built        200000 non-null  int64  
 16  year_reno         20

In [89]:
X = df_test

# Drop 'level_0' if present to match training features
if 'level_0' in X.columns:
    X = X.drop(columns=['level_0'])

X['total_baths'] = X['bath_full'] + 0.75 * X['bath_3qtr'] + 0.5 * X['bath_half']

X['view_score'] = (
    X['view_rainier'] + X['view_olympics'] + X['view_cascades'] +
    X['view_territorial'] + X['view_skyline'] + 
    X['view_sound'] + X['view_lakewash'] + 
    X['view_lakesamm']
)

drop_cols = [
    'view_rainier', 'view_olympics', 'view_cascades', 'view_territorial',
    'view_skyline', 'view_sound', 'view_lakewash', 'view_lakesamm',
]
X = X.drop(columns=drop_cols)
# drop columns that are not useful for prediction
X = X.drop(columns=['id', 'view_other', 'view_otherwater', 
                    'sale_warning', 'join_year', 'latitude', 'longitude',
                    'subdivision', 'present_use', 'sqft_1', 'sqft_fbsmt',
                    'fbsmt_grade', 'bath_full', 'bath_3qtr', 'bath_half',
                    'wfnt'])
# Encode categorical variables in the test set
encoder_1 = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = encoder_1.fit_transform(X[col].astype(str))
# Predict on the test set
y_pred_test = model.predict(X)
# Construct prediction intervals for the test set
y_pred_lower_test = y_pred_test - q
y_pred_upper_test = y_pred_test + q

In [90]:
#submission
submission_test = pd.DataFrame({
    'id': df_test['id'],
    'pi_lower': y_pred_lower_test,
    'pi_upper': y_pred_upper_test
})

In [91]:
submission_test

,id,pi_lower,pi_upper
0,200000,770755.496875,1.070335e+06
1,200001,553852.121875,8.534311e+05
2,200002,428649.684375,7.282287e+05
3,200003,197449.621875,4.970286e+05
4,200004,402364.809375,7.019438e+05
...,...,...,...
199995,399995,150076.809375,4.496558e+05
199996,399996,181987.528125,4.815665e+05
199997,399997,333926.403125,6.335054e+05
199998,399998,374807.246875,6.743863e+05


In [92]:
submission_test.to_csv('house_price_interval.csv', index=False)

In [2]:
!cd D:\Python Code VS\kaggle_compet && git init

Initialized empty Git repository in D:/Python Code VS/kaggle_compet/.git/


In [4]:
!git add .


fatal: detected dubious ownership in repository at 'D:/Python Code VS/kaggle_compet'
'D:/Python Code VS/kaggle_compet' is on a file system that does not record ownership
To add an exception for this directory, call:

	git config --global --add safe.directory 'D:/Python Code VS/kaggle_compet'
